In [73]:
import json
import pathlib

package_paths = {
    "ips": pathlib.Path(".") / "packages" / "hl7.fhir.uv.ips-1.1.0",
    "zib2017": pathlib.Path(".") / "packages" / "nictiz.fhir.nl.stu3.zib2017-2.2.20",
    "zib2020": pathlib.Path(".") / "packages" / "nictiz.fhir.nl.r4.zib2020-0.11.0-beta.1"
}

class Profile:
    def __init__(self, package, profile_name):
        self.name = profile_name
        
        if package == "ips":
            file = package_paths[package] / f"StructureDefinition-{profile_name}.json"
        else:
            file = package_paths[package] / f"{profile_name}.json"

        with open(file) as fd:
            content = json.load(fd)

        # Fish out interesting data: min and max cardinality and binding strengths
        self.elements = {}
        for el in content["differential"]["element"]:
            path = el["path"]
            data = {}
            if "min" in el:
                data["min"] = el["min"]
            if "max" in el:
                data["max"] = el["max"]
            try:
                if el["binding"]["strength"] in ["extensible", "required"]:
                    data["binding_strength"] = el["binding"]["strength"]
            except KeyError:
                pass
            if len(data):
                snapshot_el = list(filter(lambda e: e["path"] == path, content["snapshot"]["element"]))[0]
                if "min" not in data:
                    data["min"] = snapshot_el["min"]
                if "max" not in data:
                    data["max"] = snapshot_el["max"]
                if "binding_strength" not in data:
                    if "binding" in snapshot_el:
                        data["default_binding_strength"] = snapshot_el["binding"]["strength"]
                self.elements[path] = data

    def fits_in(self, other):
        findings = []
        for path in self.elements:
            if path in other.elements:
                self_el  = self.elements[path]
                other_el = other.elements[path]

                if "min" in self_el and "min" in other_el:
                    if other_el["min"] > self_el["min"]:
                        findings.append(f"{path}: required in {other.name} but optional in {self.name}")

                if "max" in self_el and "max" in other_el:
                    if other_el["max"] != "*": # if other is unrestricted we always fit
                        if self_el["max"] == "*":
                            findings.append(f"{path}: restricted to {other_el["max"]} in {other.name} but unrestricted in {self.name}")
                        else:
                            self_max = int(self_el["max"])
                            other_max = int(other_el["max"])
                            if self_max > other_max:
                                findings.append(f"{path}: restricted to {other_max} in {other.name} but to {self_max} in {self.name}")
                
                if "binding_strength" in self_el:
                    if "binding_strength" in other_el:
                        findings.append(f"{path}: {other_el["binding_strength"]} bound ValueSet in {other.name} while there's a(n) {self_el["binding_strength"]} binding in {self.name}")
                elif "binding_strength" in other_el:
                    if other_el["binding_strength"] in ["required", "extensible"]:
                        findings.append(f"{path}: {other_el["binding_strength"]} bound ValueSet in {other.name}, which is stricter than the binding in {self.name}")
        
        if len(findings):
            print(f"{self.name} does not seem to fit in {other.name}")
            for finding in findings:
                print(f"* {finding}")
        else:
            print(f"{self.name} does seem to fit in {other.name}")
                

zib_Problem2017 = Profile("zib2017", "zib-Problem")
zib_Problem2020 = Profile("zib2020", "zib-Problem")
Condition_uv_ips = Profile("ips", "Condition-uv-ips")
zib_Problem2017.fits_in(Condition_uv_ips)
Condition_uv_ips.fits_in(zib_Problem2017)
zib_Problem2020.fits_in(Condition_uv_ips)
Condition_uv_ips.fits_in(zib_Problem2020)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1973201209.py, line 73)